<a href="https://colab.research.google.com/github/rgumi/seminararbeit_src/blob/master/feb_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from scipy.stats import randint, uniform
import datetime as dt
import numpy as np
import pandas as pd 
import json
import urllib.request
pd.options.mode.chained_assignment = None

from sklearn.model_selection import train_test_split, cross_validate, RandomizedSearchCV, cross_val_score
from sklearn.metrics import f1_score, make_scorer, confusion_matrix, accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.preprocessing import MinMaxScaler
from xgboost.sklearn import XGBClassifier

In [2]:
mountpath = "/content/drive"
from google.colab import drive
drive.mount(mountpath)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# ['High School', 'University', 'Middle School', 'Professional Training', 
# 'Elementary School', 'Unknown', 'Illiterate']

def group_education(education):
  if education in ['Elementary School']:
    return 'primary'
  if education in ['Middle School', 'High School']:
    return 'secondary'
  if education in ['University', 'Professional Training']:
    return 'tertiary'
  return 'unknown'

# ['Service provider', 'Student', 'Pensioner', 'Administrator', 'Technician',
# 'Blue-collar worker', 'Self-employed', 'Unemployed', 'Manager', 'Housemaid', 'Founder', 'Unknown']

def group_job(job):
  if job in ['Service provider', 'Housemaid']:
    return 'pink-collar worker'
  if job in ['Blue-collar worker']:
    return 'blue-collar worker'
  if job in ['Administrator', 'Manager']:
    return 'white-collar worker'
  if job in ['Self-employed', 'Founder']:
    return 'independent'
  return str.lower(job)

def group_p_conv(pconv):
  if pconv == 'Successful':
    return 1
  if pconv == 'Failed':
    return 2
  return 3

def group_age(age):
  if age < 29:
    return 2
  if age < 59:
    return 3
  return 1

def group_age_q(age):
  if age < 30:
    return 1
  if age < 35:
    return 2
  if age < 42:
    return 3
  if age < 49:
    return 4
  return 5
  
def group_duration(duration):
  if duration == 0:
    return 5
  if duration < 60:
    return 4
  if duration < 300:
    return 3
  if duration < 600:
    return 2
  return 1

def group_martial_status(status):
  if status == 'single':
    return 1
  if status == 'married':
    return 2
  if status == 'divorced':
    return 3
  return 4

In [0]:
df_euribor = pd.read_csv(
    'https://raw.githubusercontent.com/rgumi/seminararbeit_src/master/refined/euribor3m_ref.csv', 
    index_col=['index'], parse_dates=['date'])

df_euribor = df_euribor[(df_euribor['date'].dt.year >= 2007)]

def get_euribor(date):
  for i in range(0, len(df_euribor)):

    if date >= df_euribor['date'].iloc[i]:
      last = df_euribor['value'].iloc[i]
      continue
    return last

In [0]:
df_eurostoxx= pd.read_csv(
    'https://raw.githubusercontent.com/rgumi/seminararbeit_src/master/refined/eurostoxx_ref.csv', 
    index_col=['index'], parse_dates=['date'])

df_eurostoxx = df_eurostoxx[(df_eurostoxx['date'].dt.year >= 2007)]

def get_eurostoxx(date):
  for i in range(0, len(df_eurostoxx)):

    if date >= df_eurostoxx['date'].iloc[i]:
      last = df_eurostoxx['value'].iloc[i]
      continue
    return last

In [0]:
df_fsi= pd.read_csv(
    'https://raw.githubusercontent.com/rgumi/seminararbeit_src/master/refined/fsi_ref.csv', 
    index_col=['index'], parse_dates=['Date'])

df_fsi = df_fsi[(df_fsi['Date'].dt.year >= 2007)]

def get_fsi(date):
  for i in range(0, len(df_fsi)):

    if date >= df_fsi['Date'].iloc[i]:
      # Possible values: [OFR FSI, Credit, Equity valuation, Safe assets, Funding, Volatility]
      last = df_fsi['OFR FSI'].iloc[i]
      continue
    return last

In [0]:
df_cpi = pd.read_csv(
    'https://raw.githubusercontent.com/rgumi/seminararbeit_src/master/refined/cpi_monthly_ref.csv', 
    index_col=['index'], parse_dates=['date'])

df_cpi = df_cpi[(df_cpi['date'].dt.year >= 2007)]

def get_cpi(date):
  for i in range(0, len(df_cpi)):

    if date >= df_cpi['date'].iloc[i]:
      last = df_cpi['value'].iloc[i]
      continue
    return last

In [0]:
df_cci = pd.read_csv(
    'https://raw.githubusercontent.com/rgumi/seminararbeit_src/master/refined/cci_monthly_ref.csv', 
    index_col=['index'], parse_dates=['date'])

df_cci = df_cci[(df_cci['date'].dt.year >= 2007)]

def get_cci(date):
  for i in range(0, len(df_cci)):

    if date >= df_cci['date'].iloc[i]:
      last = df_cci['value'].iloc[i]
      continue
      
    return last

In [0]:
ltz = {}
with urllib.request.urlopen("https://raw.githubusercontent.com/rgumi/seminararbeit_src/master/refined/leitzinsen_eu.json") as url:
    tmp_ltz = json.loads(url.read().decode())
for key in tmp_ltz.keys():
  ltz[dt.datetime.strptime(key, '%d-%m-%Y')] = tmp_ltz[key]
sorted_ltz = {k: ltz[k] for k in sorted(ltz)}

def get_leitzins(date):
  for key, val in sorted_ltz.items():
    if date >= key:
      last = val
      continue
    return last

In [10]:
dataset = pd.read_csv(
    'https://raw.githubusercontent.com/saschaschworm/big-data-and-data-science/master/datasets/prediction-challenge/dataset.csv', 
    index_col='identifier', parse_dates=['date'])

dataset.insert(len(dataset.columns) -1, "weekday", dataset.date.dt.weekday)
dataset.insert(len(dataset.columns) -1, "day", dataset.date.dt.day)
dataset.insert(len(dataset.columns) -1, "month", dataset.date.dt.month)
dataset.insert(len(dataset.columns) -1, "year", dataset.date.dt.year)
dataset.insert(len(dataset.columns) -1, "quarter", dataset.date.dt.quarter)

dataset.insert(len(dataset.columns)-1, "leitzins", dataset['date'].apply(get_leitzins))
dataset.insert(len(dataset.columns)-1, "euribor", dataset['date'].apply(get_euribor))
dataset.insert(len(dataset.columns)-1, "cci", dataset['date'].apply(get_cci))
dataset.insert(len(dataset.columns)-1, "cpi", dataset['date'].apply(get_cpi))
dataset.insert(len(dataset.columns)-1, "fsi", dataset['date'].apply(get_fsi))
dataset.insert(len(dataset.columns)-1, "eurostoxx", dataset['date'].apply(get_eurostoxx))

dataset = dataset.drop('date', axis=1)
dataset.columns

Index(['age', 'marital_status', 'education', 'job', 'credit_default',
       'housing_loan', 'personal_loan', 'communication_type',
       'n_contacts_campaign', 'days_since_last_contact', 'n_contacts_before',
       'previous_conversion', 'duration', 'weekday', 'day', 'month', 'year',
       'quarter', 'leitzins', 'euribor', 'cci', 'cpi', 'fsi', 'eurostoxx',
       'success'],
      dtype='object')

In [0]:
X, y = dataset[['month', 'education', 'job', 'age', 'previous_conversion', 'n_contacts_before',
       'days_since_last_contact', 'leitzins', 'euribor', 'n_contacts_campaign', 'marital_status',
       'credit_default', 'duration', 'housing_loan', 'cpi', 'cci', 'eurostoxx', 'fsi']], dataset['success']
y = y.apply(lambda x: 1 if x == "Yes" else 0)

#X = X[(X['job'] != 'Unknown')]

X['education'] = X['education'].apply(group_education)
X['job'] = X['job'].apply(group_job)
X['previous_conversion'] = X['previous_conversion'].apply(group_p_conv)
X['marital_status'] = X['marital_status'].apply(group_martial_status)
X['age'] = X['age'].apply(group_age_q) # group_age_q == quantile

# X['duration'] = X['duration'].apply(group_duration)

for item in ['education', 'job', 'previous_conversion',
             'credit_default', 'housing_loan']:
  try:
    encoded = pd.get_dummies(X[item], prefix=item)
    X.drop(item, axis=1, inplace=True)
    X = X.join(encoded)
  except:
    continue

scaler = MinMaxScaler()
numeric_values = ["leitzins", "euribor", 'cpi', 'cci', 'n_contacts_campaign',
                  'n_contacts_before', 'duration', 'days_since_last_contact',
                  'eurostoxx', 'fsi']

X[numeric_values] = scaler.fit_transform(X[numeric_values])

# XGBoost

In [12]:
hyperparams = {'seed': 1909, 'learning_rate': 0.01, 'min_child_weight': 0.8, 'scale_pos_weight': 8, 
               'colsample_bylevel': 0.75, 'colsample_bytree': 0.65, 'colsample_bynode': 0.75, 'max_depth': 50, 
               'n_estimators': 100, 'gamma': 0.95, 'subsample': 0.25, 'base_score': 0.5, 'reg_lambda': 1, 
               'nthread': -1, 'booster': 'gbtree', 'objective': 'binary:logistic', 'silent': True, 
               'missing': None, 'max_delta_step': 0}



model = XGBClassifier(**hyperparams)
model.fit(X, y)
print(model)
res_cv = cross_validate(model, X, y, scoring=['f1', 'accuracy', 'precision'], cv=10, return_train_score=True)
res_f1_tr = np.mean(res_cv['train_f1']) * 100
res_f1_te = np.mean(res_cv['test_f1']) * 100
print(f'Average F1 on Training and Test Set: {res_f1_tr:.2f}%/{res_f1_te:.2f}%')

# persists results for later analysis

resultJSON = {
    "date": dt.datetime.now().strftime("%d.%m.%Y, %H:%M:%S"),
    "result": f'{res_f1_tr:.2f}/{res_f1_te:.2f}',
    "hyperparams": hyperparams
}

with open(mountpath + '/My Drive/seminararbeit/results-30-01.txt', 'a') as file:
  file.write(str(resultJSON) + '\n')

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=0.75,
              colsample_bynode=0.75, colsample_bytree=0.65, gamma=0.95,
              learning_rate=0.01, max_delta_step=0, max_depth=50,
              min_child_weight=0.8, missing=None, n_estimators=100, n_jobs=1,
              nthread=-1, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=8, seed=1909,
              silent=True, subsample=0.25, verbosity=1)
Average F1 on Training and Test Set: 69.80%/63.10%


In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1909)

xgb = XGBClassifier()
xgb.fit(X_train, y_train)
xgbprd = xgb.predict(X_test)

print(confusion_matrix(y_test, xgbprd))
print(round(f1_score(y_test, xgbprd),2)*100)
XGB = (cross_val_score(estimator = xgb, X = X_train, y = y_train, cv = 10).mean())
print(XGB)
print(classification_report(y_test, xgbprd))

[[6341  215]
 [ 430  428]]
56.99999999999999
0.9141797368489877
              precision    recall  f1-score   support

           0       0.94      0.97      0.95      6556
           1       0.67      0.50      0.57       858

    accuracy                           0.91      7414
   macro avg       0.80      0.73      0.76      7414
weighted avg       0.91      0.91      0.91      7414

